# 유튜브 댓글 크롤링 (2)
### keyword: 라이프해커자청

In [6]:
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from urllib.parse import quote
from tqdm import tqdm

In [7]:
KEYWORD = "라이프해커자청"

In [8]:
v_url_list = []
driver = webdriver.Chrome('./chromedriver')
driver.maximize_window()
url = 'https://www.youtube.com/results?search_query=' + quote(KEYWORD)
driver.get(url)
time.sleep(2)

# channel_url, channel_id, channel_name 가져오기
main_link = driver.find_element_by_id("main-link")
channel_url = main_link.get_attribute("href")
c_idx = channel_url.rfind("/") + 1
channel_id = channel_url[c_idx:]
channel_name = main_link.find_element_by_id("text-container").text

# 채널 내 '동영상' 으로 이동
main_link.click()
time.sleep(1)
driver.find_element_by_xpath('//*[@id="tabsContent"]/tp-yt-paper-tab[2]/div').click()
time.sleep(1)

# 스크롤 끝까지 내리기
last_page_height = driver.execute_script("return document.documentElement.scrollHeight")
driver.execute_script("window.scrollTo(0, 350);")
while True: 
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);") 
    time.sleep(3) 
    new_page_height = driver.execute_script("return document.documentElement.scrollHeight") 
    if new_page_height == last_page_height: 
        break 
    last_page_height = new_page_height

lis = driver.find_elements_by_id('thumbnail')

for li in lis:
    video_url = li.get_attribute("href")
    v_url_list.append(video_url)

driver.close()

In [9]:
print(len(v_url_list), "\n", v_url_list)

49 
 ['https://www.youtube.com/watch?v=AUCKb5b5UNs', 'https://www.youtube.com/watch?v=1Jgi2zSavT0', 'https://www.youtube.com/watch?v=4Yfu0-XHGgU', 'https://www.youtube.com/watch?v=SR2SWfIXt7s', 'https://www.youtube.com/watch?v=WBlWGf70H1g', 'https://www.youtube.com/watch?v=7qBpW5hHLho&list=PLdaXIYLRupw5LlrPXnDeBzbI8N8j6tnei', 'https://www.youtube.com/watch?v=KNqywKwNUSA', 'https://www.youtube.com/watch?v=PPAGzwOw7So', 'https://www.youtube.com/watch?v=PTJhFN9ziEE', 'https://www.youtube.com/watch?v=0IYxVpuL-Qw', 'https://www.youtube.com/watch?v=GPuSUBHUDFA', 'https://www.youtube.com/watch?v=F0ZHHYl6Cyg', 'https://www.youtube.com/watch?v=fgdSQqDa7sE', 'https://www.youtube.com/watch?v=MLwDArWupOw', 'https://www.youtube.com/watch?v=f2hYXC1A5Nc', 'https://www.youtube.com/watch?v=1dsWprRplgU', 'https://www.youtube.com/watch?v=9bvSBMONdZc', 'https://www.youtube.com/watch?v=imk6pmLy2zE', 'https://www.youtube.com/watch?v=PcFdkiYnXTI', 'https://www.youtube.com/watch?v=qPgvZrJwVHs', 'https://www.y

In [11]:
comment_list = []; err = 0
driver = webdriver.Chrome('./chromedriver')
driver.maximize_window()
for url in tqdm(v_url_list):
    if url == None:
        continue
    else:
        driver.get(url)
        time.sleep(1)

        # 스크롤 끝까지 내리기
        last_height = driver.execute_script("return document.documentElement.scrollHeight")
        driver.execute_script("window.scrollTo(0, 350);")
        time.sleep(2)
        while True:
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
            time.sleep(1.5)
            new_height = driver.execute_script("return document.documentElement.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        # 댓글 수집
        contents = driver.find_element_by_class_name("style-scope ytd-item-section-renderer")
        try:
            comments = contents.find_elements_by_tag_name('ytd-expander')
            for comment in comments:
                text = comment.find_element_by_tag_name('div').text
                comment_list.append(text)
        except:
            err += 1
            continue

print(f'Error Count: {err}')
driver.close()

100%|██████████| 49/49 [46:05<00:00, 56.44s/it]
Error Count: 0


In [12]:
len(comment_list)

15428

### 댓글 원본 저장(전처리 X)

In [13]:
import pandas as pd 
df = pd.DataFrame({
    'comments': comment_list
})
df.head()

,comments
0,ㅜㅜ마지막 부분 보는데 주책맞게 살짝 울었네요. 아무튼 사업에 대한 스킬적인 것 보...
1,와 진짜 끝까지 집중해서 들었어요 ㅠㅠㅠㅠ 제가 지금까지 가지고 있던 고민을 해결할...
2,세계를 최적화=홍익인간 (널리 인간을 이롭게하라)
3,5:46 최적화를 하는 사람이 많을 수록 그 국가는 부유해진다는 말이 뭔가 소름돋네...
4,내가 와닿고 내것으로 습득할때까지 계속 보러와야겠어요..


In [14]:
df.to_csv(f'./data/youtube_channel={KEYWORD}_org.txt', sep='\t', index=False)

### 데이터 전처리

In [15]:
import pandas as pd 
df_test = pd.read_table(f'./data/youtube_channel={KEYWORD}_org.txt')
df_test.head()

,comments
0,ㅜㅜ마지막 부분 보는데 주책맞게 살짝 울었네요. 아무튼 사업에 대한 스킬적인 것 보...
1,와 진짜 끝까지 집중해서 들었어요 ㅠㅠㅠㅠ 제가 지금까지 가지고 있던 고민을 해결할...
2,세계를 최적화=홍익인간 (널리 인간을 이롭게하라)
3,5:46 최적화를 하는 사람이 많을 수록 그 국가는 부유해진다는 말이 뭔가 소름돋네...
4,내가 와닿고 내것으로 습득할때까지 계속 보러와야겠어요..


In [16]:
df_test.shape

(15428, 1)

In [17]:
df_test['comments'] = df_test['comments'].str.replace("[^a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣 ]","")
df_test.head()

<ipython-input-17-4884171cc048>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_test['comments'] = df_test['comments'].str.replace("[^a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣 ]","")


,comments
0,ㅜㅜ마지막 부분 보는데 주책맞게 살짝 울었네요 아무튼 사업에 대한 스킬적인 것 보다...
1,와 진짜 끝까지 집중해서 들었어요 ㅠㅠㅠㅠ 제가 지금까지 가지고 있던 고민을 해결할...
2,세계를 최적화홍익인간 널리 인간을 이롭게하라
3,최적화를 하는 사람이 많을 수록 그 국가는 부유해진다는 말이 뭔가 소름돋네요 좋은...
4,내가 와닿고 내것으로 습득할때까지 계속 보러와야겠어요


In [18]:
# 'comments' 값 공백 여부 확인
drop_idx = df_test[df_test.comments == ""].index
print(len(drop_idx), drop_idx)

97 Int64Index([  929,   967,  1022,  1050,  1295,  1321,  1767,  2713,  2791,
             2994,  3457,  3503,  3507,  3510,  3512,  3524,  3542,  3571,
             3906,  4021,  4309,  4315,  4340,  4708,  4713,  4717,  4851,
             4941,  5891,  5929,  5984,  6012,  6257,  6283,  7066,  7107,
             7119,  7128,  7135,  7140,  7522,  7542,  7544,  7961,  7970,
             8634,  8642,  8645,  8677,  8691,  8711,  9061,  9066,  9070,
             9872,  9958,  9971, 10042, 10052, 10082, 10150, 11009, 11086,
            11107, 11110, 11164, 11176, 11182, 11943, 12005, 12201, 12428,
            12446, 12456, 12650, 12966, 13081, 13365, 13369, 13414, 13426,
            13583, 13651, 14059, 14105, 14109, 14112, 14114, 14126, 14144,
            14173, 14342, 14348, 14373, 15124, 15142, 15410],
           dtype='int64')


In [19]:
# 공백 값을 가지고 있는 해당 인덱스 행 제거
df_test.drop(drop_idx, inplace=True)
df_test.shape

(15331, 1)

In [20]:
# null값 여부 확인
df_test.isnull().sum()

comments    0
dtype: int64

In [21]:
df_test.head()

,comments
0,ㅜㅜ마지막 부분 보는데 주책맞게 살짝 울었네요 아무튼 사업에 대한 스킬적인 것 보다...
1,와 진짜 끝까지 집중해서 들었어요 ㅠㅠㅠㅠ 제가 지금까지 가지고 있던 고민을 해결할...
2,세계를 최적화홍익인간 널리 인간을 이롭게하라
3,최적화를 하는 사람이 많을 수록 그 국가는 부유해진다는 말이 뭔가 소름돋네요 좋은...
4,내가 와닿고 내것으로 습득할때까지 계속 보러와야겠어요


In [22]:
print(f'전처리 후 샘플의 개수: {len(df_test)}')

전처리 후 샘플의 개수: 15331


In [23]:
df_test.to_csv(f'./result/youtube_channel={KEYWORD}_res.txt', sep='\t', index=False)